# Import Libraries

In [1]:
from deploy_baseline import ResearchPotatoWrapper
from tqdm import trange
import json
import re
import random
#import numpy as np
#import torch as th
#from torch import nn
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
#from pyvirtualdisplay import Display
#from sklearn.cluster import KMeans

from os import listdir,makedirs
from os.path import join
from collections import defaultdict

import logging



logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.


/home/olli/gits/Minecraft72/venv/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [2]:


# place models here or edit folder name. All models in this folder will be evaluated.
modeldir = 'research_baseline_models'

#collect names
modeldirs = [join(modeldir,d) for d in listdir(modeldir)]


In [3]:
# how many episodes per model?
number_of_episodes = 25
# how many steps per episode?
episode_len = 2000

# random seeds. keep it consistent
seeds = [3649, 7327 ,3779, 4914, 4693 ,1999 ,1523 ,3072 ,6983, 1100  ,723 , 716, 5443 ,5903,
 9829 ,4407 ,7370, 8878, 8745, 5988, 3247 ,8821 , 728, 8479 ,8061]

In [4]:

def eval(modeldir):
    
    # split at '/'. Only works on linux i think!
    modelname = modeldir.split('/')[1]
    
    ## use regex to get latent-pic-dimension. will fail for arbitrary model names
    latent_dimension_reg = re.compile("latent-pic-dimension=\d{1}")
    print(modelname)



    # not robust, be careful!
    latent_dim = latent_dimension_reg.search(modelname)

    # if not parsable skip!
    if not latent_dim :
        print(f"skipping {modelname}, couldnt parse name!")
        return None
        
    latent_dim= latent_dim.group(0)
    latent_dim = int(latent_dim[-1])

    
    # load model
    potato = ResearchPotatoWrapper(modeldir,latent_dim)
    
    # start env
    env = gym.make('MineRLTreechopVectorObf-v0')
    
    # start recorder. This could possibly be improved
    env = Recorder(env, join('./video',modelname), fps=24)
    
    # collect data for all tested models
    episode_rewards = defaultdict(lambda: [])
    

    
    for episode in trange(number_of_episodes):
        
        #makedirs(join('./video',modelname,str(episode)),exist_ok=True)

        env.seed(seeds[episode])
        obs = env.reset()
        
        done = False
        total_reward = 0
        steps = 0

        while not done:
             
            # deployment is easy!
            minerl_action = potato.predict_action(obs)
            
            # step from prediction 
            obs, reward, done, info = env.step(minerl_action)
            total_reward += reward
            steps += 1
            
            # break if episode len is reached.
            if steps >= episode_len:
                break
        
        # save vid?
        env.release()
        
        # you can uncomment this on colab. Fails locally and on kaggle.
        #env.play()
        
        print(f'Episode #{episode + 1} reward: {total_reward}\t\t episode length: {steps}\n')
        
        # save globally
        episode_rewards['reward'].append(total_reward)
        episode_rewards['steps'].append(steps)
        
    # close env. not really working ...    
    env.close()
    makedirs(join('rewards'),exist_ok=True)
    print("Saving rewards")
    
    # dump rewards as .json. can be loaded with json.load(open("filename"m'r)) example in rewards folder.
    json.dump(episode_rewards,open(join('rewards',f'{modelname}_rewards.json'),'w'))
    
    return episode_rewards

In [5]:
for modeldir in modeldirs:
    rewards = []
    ev = eval(modeldir)
    if ev:
        rewards.append(ev)
    

old
skipping old, couldnt parse name!
window-before=40_window-after=20_latent-pic-dimension=4_epochs=8_clusters=70


/home/olli/gits/Minecraft72/venv/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'minerl.utils.process_watcher' found in sys.modules after import of package 'minerl.utils', but prior to execution of 'minerl.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
  0%|                                                                                                        | 0/25 [00:00<?, ?it/s]OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  4%|███▋                                                                                         | 1/25 [03:00<1:12:19, 180.83s/it]

Episode #1 reward: 4.0		 episode length: 2000



OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  8%|███████▍                                                                                     | 2/25 [06:00<1:09:00, 180.03s/it]

Episode #2 reward: 13.0		 episode length: 2000



...

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [55:15<00:00, 132.63s/it]

Episode #25 reward: 0.0		 episode length: 2000



Saving rewards
window-before=40_window-after=20_latent-pic-dimension=4_epochs=4_clusters=70


/home/olli/gits/Minecraft72/venv/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'minerl.utils.process_watcher' found in sys.modules after import of package 'minerl.utils', but prior to execution of 'minerl.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


KeyboardInterrupt: 

In [ ]:
json.dump(rewards,open('rewards.txt','w'))